In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [6]:
with open('../data/k2pandc_2025.10.03_13.31.02.csv') as k2panda, open('../data/cumulative_2025.10.03_13.30.13.csv') as cumulative, open('../data/TOI_2025.10.03_12.20.22.csv') as toi:
    k2panda_df = pd.read_csv(k2panda)
    cumulative_df = pd.read_csv(cumulative)
    toi_df = pd.read_csv(toi)
df

,pl_name,hostname,default_flag,disposition,disp_refname,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,...,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
0,BD+20 594 b,BD+20 594,0,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,K2,...,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249,2018-04-25,2018-03,2018-02-15
1,BD+20 594 b,BD+20 594,0,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,K2,...,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249,2018-04-25,2016-10,2016-07-28
2,BD+20 594 b,BD+20 594,1,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,K2,...,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249,2018-04-25,2017-03,2018-04-26
3,EPIC 201111557.01,EPIC 201111557,1,CANDIDATE,Livingston et al. 2018,1,0,Transit,2018,K2,...,-0.046,9.220,0.019,-0.019,11.39950,0.001307,-0.001307,2018-08-02,2018-08,2018-08-02
4,EPIC 201111557.01,EPIC 201111557,0,CANDIDATE,Livingston et al. 2018,1,0,Transit,2018,K2,...,-0.046,9.220,0.019,-0.019,11.39950,0.001307,-0.001307,2018-02-15,2018-03,2018-02-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,WASP-85 A b,WASP-85 A,0,CONFIRMED,Mo&#x10D;nik et al. 2016,2,1,Transit,2016,K2,...,-0.010,8.733,0.026,-0.026,10.62410,0.000959,-0.000959,2019-09-05,2019-09,2019-09-05
4000,Wolf 503 b,Wolf 503,1,CONFIRMED,Peterson et al. 2018,1,1,Transit,2018,K2,...,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337,2022-05-23,2021-12,2022-05-23
4001,Wolf 503 b,Wolf 503,0,CONFIRMED,Peterson et al. 2018,1,1,Transit,2018,K2,...,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337,2018-09-04,2018-11,2018-09-06
4002,Wolf 503 b,Wolf 503,0,CONFIRMED,Peterson et al. 2018,1,1,Transit,2018,K2,...,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337,2023-04-17,2023-04,2023-04-17
